In [1]:
# import libraries:
import pandas as pd 
import numpy as np 
import os 

In [2]:
# set path: 
os.chdir("/Users/carlosswanton/Desktop/Proyecto - Py/beer_data")
# get a list of the files in the directory:
data = os.listdir()
# create an empty dataframe: 
df = pd.DataFrame()
# count files in directory:
count = 0 
for database in data: 
    count+=1 
    print(count) 
print("Number of files in the directory: " + str(count))

1
2
3
4
5
6
Number of files in the directory: 6


In [3]:
# loop thru the directory to make a concat of all databases:
timer = 6
for database in data:
    df_temp = pd.read_excel("/Users/carlosswanton/Desktop/Proyecto - Py/beer_data/"+database)
    df = pd.concat([df, df_temp]) 
    timer -=  1
    print("Databases left to concat: " + str(timer))
print("Done")

Databases left to concat: 5
Databases left to concat: 4
Databases left to concat: 3
Databases left to concat: 2
Databases left to concat: 1
Databases left to concat: 0
Done


In [4]:
# dislay all columns of dataframe:
pd.set_option('display.max_columns', None)

In [5]:
# shape of df:
df.shape

(8950, 86)

In [6]:
# start cleaning the dataset, first the column names and drop the ones you do not need. 
df.columns = df.columns.str.lower()
df.reset_index(inplace = True, drop= True)

In [7]:
# create a subset of the original dataframe: 
df = df[["año", "mes", "ruc", "razon social", "desc_comer", "marca_comercial",  "marcas", "ciudad_embarque", "pais_embarque", "pais_origen", 
    "unidades", "tipo_unidad", "kilos_neto", "fob", "flete", "seguro", "cif", "fob_unitario", "incoterm"]] 

In [8]:
# renaming columns
df.rename(columns = {"razon social" : "razon_social"}, inplace = True)

In [9]:
df.head(1)

,año,mes,ruc,razon_social,desc_comer,marca_comercial,marcas,ciudad_embarque,pais_embarque,pais_origen,unidades,tipo_unidad,kilos_neto,fob,flete,seguro,cif,fob_unitario,incoterm
0,2018,1,190007510001,ALMACENES JUAN ELJURI CIA. LTDA.,CERVEZA,SOL,SOL,NaN,NETHERLANDS,NETHERLANDS,43908.0,UNIDADES,44346.0,55273.68,0.0,476.78,55750.46,9.97,CFR


In [10]:
# clean dataframe: look for missing values and bad data.

- There are two columns with the name of the brand and one column with the commercial description. 
- I will be exploring which column to choose in order to explore the data. 


In [11]:
df["marcas"].isna().sum()

414

In [12]:
df["marcas"].value_counts(dropna= False).head(5)

CORONA           1737
CORONA EXTRA     1070
STELLA ARTOIS     933
BUDWEISER         424
NaN               414
Name: marcas, dtype: int64

In [13]:
df["marca_comercial"].isna().sum()

415

In [14]:
df["marca_comercial"].value_counts(dropna= False).head(5)

CORONA           1737
CORONA EXTRA     1070
STELLA ARTOIS     933
BUDWEISER         424
NaN               415
Name: marca_comercial, dtype: int64

In [15]:
# they are similar. i will use "marcas" because has one less mising value.

- is it possible to fill the missing values of the beer brand (marcas) with the commercial description?

In [16]:
dummy = df[["desc_comer","marcas","marca_comercial"]]

In [17]:
filt = (dummy["marcas"].isna())
nan = dummy.loc[filt]

In [18]:
# nan is the new datadrame that we need to understand in order to put the beer brands in marcas. 

In [19]:
nan.head()

,desc_comer,marcas,marca_comercial
7,PILSENER BEER MALTA BEVERAGE,NaN,NaN
8,CLUB BEER MALTA BEVERAGE,NaN,NaN
9,CLUB BEER MALTA BEVERAGE,NaN,NaN
32,PONY MALTA,NaN,NaN
99,BOTELLAS DE CERVEZA,NaN,NaN


In [20]:
nan["desc_comer"].value_counts().head()

CERVEZA                            32
CERV HEINEKEN OW 4X6 330cc         32
PILSENER BEER MALTA BEVERAGE       17
CERVEZA PILSENER 600CM             13
PILSEN TRUJILLO LATA 355 ML 6X1    13
Name: desc_comer, dtype: int64

In [21]:
nan1 = nan.copy() # experiment with copying the commercial description and filling missing values.

In [22]:
nan1['marcas'].fillna(nan1['desc_comer'], inplace = True)

In [23]:
nan1

,desc_comer,marcas,marca_comercial
7,PILSENER BEER MALTA BEVERAGE,PILSENER BEER MALTA BEVERAGE,NaN
8,CLUB BEER MALTA BEVERAGE,CLUB BEER MALTA BEVERAGE,NaN
9,CLUB BEER MALTA BEVERAGE,CLUB BEER MALTA BEVERAGE,NaN
32,PONY MALTA,PONY MALTA,NaN
99,BOTELLAS DE CERVEZA,BOTELLAS DE CERVEZA,NaN
...,...,...,...
8873,CLUB BEER MALTA BEVERAGE,CLUB BEER MALTA BEVERAGE,NaN
8874,CLUB BEER MALTA BEVERAGE,CLUB BEER MALTA BEVERAGE,NaN
8943,CERVEZA BIELA 343CC X 24 BOTELLAS,CERVEZA BIELA 343CC X 24 BOTELLAS,NaN
8944,CERVEZA BIELA 343CC SIX PACK,CERVEZA BIELA 343CC SIX PACK,NaN


In [24]:
# back to our dataframe

In [25]:
df["marcas"].fillna(df["desc_comer"], inplace = True)

In [26]:
df["marcas"].value_counts(dropna=False)

CORONA                            1737
CORONA EXTRA                      1070
STELLA ARTOIS                      933
BUDWEISER                          424
HEINEKEN                           334
                                  ... 
BAVARIA & CIA S.C.A.                 1
PREMIUM LAGER                        1
LATAS PACK 269ML X 12                1
LATA PACKS 473 ML X 12               1
CERVEZA ARTESANAL SIERRA NEGRA       1
Name: marcas, Length: 368, dtype: int64

In [27]:
df["marcas"].isna().sum()

0

In [28]:
df.drop(columns = ["desc_comer", "marca_comercial"], inplace = True) 

In [29]:
# we now have to group the beer brands, because they are written differently sometimes.

In [30]:
df["marcas"].value_counts()
 # CORONA AND CORONA EXTRA ARE THE SAME. We need to correctly group beers.

CORONA                            1737
CORONA EXTRA                      1070
STELLA ARTOIS                      933
BUDWEISER                          424
HEINEKEN                           334
                                  ... 
BAVARIA & CIA S.C.A.                 1
PREMIUM LAGER                        1
LATAS PACK 269ML X 12                1
LATA PACKS 473 ML X 12               1
CERVEZA ARTESANAL SIERRA NEGRA       1
Name: marcas, Length: 368, dtype: int64

In [31]:
x = df["marcas"].value_counts()
type(x)

pandas.core.series.Series

In [32]:
# check for names and duplicates. (Send to Excel and get a better grip of the names).
os.chdir("/Users/carlosswanton/Desktop/Proyecto - Py/") 
x.to_excel("Brands_list.xlsx")

In [33]:
df["marcas"].replace({"CORONA EXTRA": "CORONA"}, inplace = True)
df["marcas"].replace({"CORONITA": "CORONA"}, inplace = True)
df["marcas"].replace({"BECK": "BECKS"}, inplace = True)
df["marcas"].replace({"BECKS": "BECKS"}, inplace = True)
df["marcas"].replace({"BECK S": "BECKS"}, inplace = True)
df["marcas"].replace({"CERV HEINEKEN OW 4X6 330cc": "HEINEKEN"}, inplace = True)
df["marcas"].replace({"CERV HEINEKEN OW 4X6 330cc": "HEINEKEN"}, inplace = True)
df["marcas"].replace({"STELLA": "STELLA ARTOIS"}, inplace = True)
df["marcas"].replace({"CORONA EXTRA SLEEK": "CORONA"}, inplace = True)
df["marcas"].replace({"CORONITA EXTRA": "CORONA"}, inplace = True)
df["marcas"].replace({"HEINEKEN LAGER BEER": "HEINEKEN"}, inplace = True)
df["marcas"].replace({"CERVEZA SOL": "SOL"}, inplace = True)
df["marcas"].replace({"CERVEZA HEINEKEN": "HEINEKEN"}, inplace = True)
df["marcas"].replace({"BECKS 269ML X24": "BECKS"}, inplace = True)
df["marcas"].replace({"CERV HEINEKEN OW 4X6 - 330CC": "HEINEKEN"}, inplace = True)
df["marcas"].replace({"ORIGINAL OETTINGER": "OETTINGER"}, inplace = True)
df["marcas"].replace({"OETTTTINGER": "OETTINGER"}, inplace = True)
df["marcas"].replace({"ROYAL DUTCH POST HORN": "ROYAL DUTCH"}, inplace = True)
df["marcas"].replace({"ROYAL DUTCH POS HORN": "ROYAL DUTCH"}, inplace = True)
df["marcas"].replace({"WARSTEINER BRAUEREI": "WARSTEINER"}, inplace = True)

In [34]:
df["marcas"].value_counts(normalize=True).sort_values(ascending=False).head(20).sum()
# I have checked the 20 first brandas, they represent more than 77 percent of orders.

0.7627932960893854

In [35]:
df["marcas"].value_counts().sort_values(ascending=False).head(10)

CORONA             3055
STELLA ARTOIS       953
BUDWEISER           424
HEINEKEN            386
WIERQUER            257
OETTINGER           246
ROYAL DUTCH         229
BECKS               212
MODELO ESPECIAL     135
ERDINGER            107
Name: marcas, dtype: int64

#### format the dataframe

In [36]:
df["razon_social"] = df["razon_social"].str.lower()
df["marcas"] = df["marcas"].str.lower()
df["pais_embarque"] = df["pais_embarque"].str.lower()
df["pais_origen"] = df["pais_origen"].str.lower()

In [37]:
df.head(3)

,año,mes,ruc,razon_social,marcas,ciudad_embarque,pais_embarque,pais_origen,unidades,tipo_unidad,kilos_neto,fob,flete,seguro,cif,fob_unitario,incoterm
0,2018,1,190007510001,almacenes juan eljuri cia. ltda.,sol,NaN,netherlands,netherlands,43908.0,UNIDADES,44346.0,55273.68,0.0,476.78,55750.46,9.97,CFR
1,2018,1,190148920001,fraganlicor cia. ltda.,heineken,NaN,netherlands,netherlands,27648.0,UNIDADES,27924.0,33776.64,0.0,293.07,34069.71,7.33,CFR
2,2018,1,190148920001,fraganlicor cia. ltda.,heineken,NaN,netherlands,netherlands,13782.0,UNIDADES,13920.0,16837.01,0.0,146.09,16983.10,7.33,CFR


In [38]:
# where do beer comes from?

In [39]:
df["ciudad_embarque"].value_counts(normalize=True, dropna = False).head()

NaN                                       0.958212
BALBOA                                    0.009721
MANZANILLO                                0.005810
CALLAO                                    0.004693
SANTIAGO-ARTURO MERINO BENITEZ AIRPORT    0.002570
Name: ciudad_embarque, dtype: float64

In [40]:
df.drop(columns = ["ciudad_embarque"], inplace=True) # no info: 96 percent are missing values

In [41]:
# country of shipping

In [42]:
df["pais_embarque"].value_counts(dropna=False)

mexico             2751
germany            1224
belgica            1211
colombia            958
netherlands         800
spain               496
united states       298
brazil              276
NaN                 231
belgium             145
peru                137
italy                92
polonia              57
united kingdom       52
chile                34
poland               30
dinamarca            28
china                25
republica checa      24
espa                 19
irlanda              17
portugal             13
puerto rico           8
panama                6
france                5
argentina             4
estados unidos        3
sudafrica             2
hungria               1
czech republic        1
paraguay              1
new zealand           1
Name: pais_embarque, dtype: int64

In [43]:
filt = (df["pais_embarque"].isna()) 
df.loc[filt].head() 

,año,mes,ruc,razon_social,marcas,pais_embarque,pais_origen,unidades,tipo_unidad,kilos_neto,fob,flete,seguro,cif,fob_unitario,incoterm
2370,2022,7,990017514001,tiendas industriales asociadas tia s. a.,cerveza konigsb pils 4.6,NaN,NaN,18817.92,61-LITRO,0.00,10243.150,0.00,0.00,0.000,0.000000,NaN
2540,2022,8,190148920001,fraganlicor cia. ltda.,estrella damm,NaN,españa,8624.88,61-LITRO,8697.33,8739.720,6214.52,120.93,8860.650,8.025455,FCA
2558,2022,8,990023549001,cerveceria nacional cn s.a.,corona,NaN,mexico,22100.88,61-LITRO,22334.34,20030.222,827.00,65.00,20095.222,7.721751,FAS
2559,2022,8,990023549001,cerveceria nacional cn s.a.,modelo especial,NaN,mexico,14313.60,61-LITRO,12398.40,18695.840,827.00,18.50,18714.340,11.128476,FAS
2560,2022,8,990023549001,cerveceria nacional cn s.a.,modelo especial,NaN,mexico,14313.60,61-LITRO,12398.40,18695.840,827.00,18.50,18714.340,11.128476,FAS


In [44]:
# i have to choose country of origin (not country of shipping) because we need to know where did the beer was manufactured.

In [45]:
df["pais_origen"].value_counts(dropna=False) #??? why does ecuador appears 383 times?

mexico                         2797
belgica                        1389
germany                        1228
colombia                        987
netherlands                     818
spain                           481
ecuador                         383
brazil                          281
united states                   125
polonia                          94
italia                           74
reino unido de gran bretaña      51
peru                             49
dinamarca                        28
china                            26
republica checa                  25
espa                             19
alemania                         18
irlanda                          17
portugal                         15
NaN                              12
paises bajos                     10
españa                            7
estados unidos                    6
argentina                         4
france                            3
chile                             1
paraguay                    

In [46]:
df["pais_origen"].value_counts(normalize=True).head(7) 

mexico         0.312934
belgica        0.155404
germany        0.137391
colombia       0.110427
netherlands    0.091519
spain          0.053815
ecuador        0.042851
Name: pais_origen, dtype: float64

- Ecuador represents 4 percent of the country of origin?

In [47]:
df["marcas"].value_counts(normalize=True).head(7)

corona           0.341341
stella artois    0.106480
budweiser        0.047374
heineken         0.043128
wierquer         0.028715
oettinger        0.027486
royal dutch      0.025587
Name: marcas, dtype: float64

In [48]:
df["pais_origen"].replace({"germany": "alemania"}, inplace = True)
df["pais_origen"].replace({"netherlands": "paises bajos"}, inplace = True)
df["pais_origen"].replace({"paises bajos": "países bajos"}, inplace = True)
df["pais_origen"].replace({"spain": "españa"}, inplace = True)
df["pais_origen"].replace({"espa": "españa"}, inplace = True)
df["pais_origen"].replace({"brazil": "brasil"}, inplace = True)
df["pais_origen"].replace({"united states": "estados unidos"}, inplace = True)
df["pais_origen"].replace({"reino unido de gran bretaña": "reino unido"}, inplace = True)
df["pais_origen"].replace({"espa": "españa"}, inplace = True)

In [49]:
filt = (df["pais_origen"]== "ecuador") 
ec = df.loc[filt]

In [50]:
ec.tail()

,año,mes,ruc,razon_social,marcas,pais_embarque,pais_origen,unidades,tipo_unidad,kilos_neto,fob,flete,seguro,cif,fob_unitario,incoterm
8872,2017,11,301256996001,jimbo campoverde fredy paul,pilsener beer malta beverage,mexico,ecuador,9580.459961,UNIDADES,14797.599609,23927.04,0.0,0.0,0.0,0.0,NaN
8873,2017,11,301256996001,jimbo campoverde fredy paul,club beer malta beverage,mexico,ecuador,1946.280029,UNIDADES,3006.149902,6076.00,0.0,0.0,0.0,0.0,NaN
8874,2017,11,301256996001,jimbo campoverde fredy paul,club beer malta beverage,mexico,ecuador,1745.930054,UNIDADES,1865.939941,5227.92,0.0,0.0,0.0,0.0,NaN
8943,2017,12,991402012001,dinaper s.a.,cerveza biela 343cc x 24 botellas,united states,ecuador,13994.000000,UNIDADES,2760.000000,4460.00,0.0,0.0,0.0,0.0,NaN
8944,2017,12,991402012001,dinaper s.a.,cerveza biela 343cc six pack,united states,ecuador,6997.000000,UNIDADES,1380.000000,2250.00,0.0,0.0,0.0,0.0,NaN


In [51]:
filte = (df["pais_origen"]== "colombia") 
colo = df.loc[filte]
colo["marcas"].value_counts(normalize=True)
# colombia produces mexican beer like corona. 

corona                                            0.584600
budweiser                                         0.284701
bavaria                                           0.047619
miller                                            0.027356
bavaria y cia s.c.a.                              0.021277
cerveceria bbc de la sabana sas                   0.012158
cerveceria del valle                              0.005066
bavaria s.a                                       0.004053
stella artois                                     0.003040
sin marca                                         0.002026
tutaina                                           0.002026
bavaria & cia s.c.a.                              0.001013
andina                                            0.001013
latas de cervezas poker 4.5% grados de alcohol    0.001013
para la industria cervecera                       0.001013
ambev                                             0.001013
miller gd exp tw 355cc 6x4                        0.0010

In [52]:
df["pais_origen"].replace({"ecuador": "otros"}, inplace = True)

In [53]:
df["pais_origen"].value_counts(normalize=True, dropna = False)

mexico             0.312514
belgica            0.155196
alemania           0.139218
colombia           0.110279
países bajos       0.092514
españa             0.056648
otros              0.042793
brasil             0.031397
estados unidos     0.014637
polonia            0.010503
italia             0.008268
reino unido        0.005698
peru               0.005475
dinamarca          0.003128
china              0.002905
republica checa    0.002793
irlanda            0.001899
portugal           0.001676
NaN                0.001341
argentina          0.000447
france             0.000335
chile              0.000112
paraguay           0.000112
hungria            0.000112
Name: pais_origen, dtype: float64

In [54]:
os.chdir("/Users/carlosswanton/Desktop/Proyecto - Py/Cleaned_data/")
df.to_excel("cheers.xlsx")